In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Equilibrage 
from sklearn.utils import resample

# Modele
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_parquet('flights_data.parquet')

In [3]:
df.dtypes

MONTH                  int64
DAY_OF_MONTH          object
UNIQUE_CARRIER        object
FL_NUM                object
ORIGIN_CITY_NAME      object
ORIGIN_STATE_NM       object
DEST_CITY_NAME        object
CRS_DEP_TIME         float64
DISTANCE             float64
ORIGIN_STATE_ABR      object
DEST_STATE_ABR        object
ORIGIN_AIRPORT_ID     object
DEST_AIRPORT_ID       object
DEP_TIME_BLK          object
ARR_TIME_BLK          object
HISTORICAL_DELAY     float64
ARR_DELAY            float64
dtype: object

In [4]:
index_to_drop = df[df['FL_NUM'] == '582700-1759'].index

# Supprimer la ligne
df = df.drop(index_to_drop)

In [5]:
# Convertir les colonnes en int
df['DAY_OF_MONTH'] = df['DAY_OF_MONTH'].astype(int)
df['FL_NUM'] = df['FL_NUM'].astype(int)
df['ORIGIN_AIRPORT_ID'] = df['ORIGIN_AIRPORT_ID'].astype(float).astype(int)
df['DEST_AIRPORT_ID'] = df['DEST_AIRPORT_ID'].astype(float).astype(int)

# Afficher les types de données après conversion
print(df.dtypes)

MONTH                  int64
DAY_OF_MONTH           int64
UNIQUE_CARRIER        object
FL_NUM                 int64
ORIGIN_CITY_NAME      object
ORIGIN_STATE_NM       object
DEST_CITY_NAME        object
CRS_DEP_TIME         float64
DISTANCE             float64
ORIGIN_STATE_ABR      object
DEST_STATE_ABR        object
ORIGIN_AIRPORT_ID      int64
DEST_AIRPORT_ID        int64
DEP_TIME_BLK          object
ARR_TIME_BLK          object
HISTORICAL_DELAY     float64
ARR_DELAY            float64
dtype: object


# Equilibrage

In [6]:
# Undersampling

In [7]:
# Créer la colonne binaire 'delayed'
df['delayed'] = df['ARR_DELAY'].apply(lambda x: 1 if x < 0 else 0)

# Afficher les données initiales
print("Données initiales :")
print(df['delayed'].value_counts())

Données initiales :
1    3534105
0    2078272
Name: delayed, dtype: int64


In [8]:
# Séparer les classes majoritaires et minoritaires
classe_majoritaire = df[df.delayed == 0]
classe_minoritaire = df[df.delayed == 1]

# Déterminer le nombre d'échantillons à sous-échantillonner
n_samples = min(len(classe_majoritaire), len(classe_minoritaire))

# Effectuer l'undersampling de la classe majoritaire
classe_majoritaire_under = resample(classe_majoritaire, 
                                    replace=False,    
                                    n_samples=n_samples,  
                                    random_state=42)  

# Effectuer l'undersampling de la classe minoritaire
classe_minoritaire_under = resample(classe_minoritaire, 
                                    replace=False,    
                                    n_samples=n_samples,  
                                    random_state=42)

# Combiner les classes minoritaires et majoritaires sous-échantillonnées
df_under = pd.concat([classe_majoritaire_under, classe_minoritaire_under])

# Afficher les données après undersampling
print("\nDonnées après undersampling :")
print(df_under['delayed'].value_counts())


Données après undersampling :
0    2078272
1    2078272
Name: delayed, dtype: int64


# Modèles

## Classification

### DummyClassifier

In [9]:
# Sélection des colonnes à exclure
columns_to_exclude = ['delayed']

# Séparation des features (X) et de la cible (y)
X = df_under.drop(columns=columns_to_exclude)
y = df_under['delayed']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sélection des caractéristiques numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Prétraitement des données
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Créer le pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DummyClassifier(strategy="most_frequent"))
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Évaluer le modèle
accuracy = pipeline.score(X_test, y_test)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.4997


### RandomForestClassifier

In [ ]:
# Sélection des colonnes à exclure
columns_to_exclude = ['delayed', 'ARR_DELAY']

# Séparation des features (X) et de la cible (y)
X = df_under.drop(columns=columns_to_exclude)
y = df_under['delayed']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sélection des caractéristiques numériques et catégorielles
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Pipeline pour les transformations préliminaires
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Pipeline final combinant le prétraitement et le modèle
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=5, random_state=42))
])

# Entraîner le pipeline
pipeline.fit(X_train, y_train)

# Évaluer le modèle
accuracy = pipeline.score(X_test, y_test)
print(f"Accuracy: {accuracy:.4f}")